# Simple MNL: Comparison with R's mlogit package

In [ ]:
import os

# Remove GPU use
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys

sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np

from choice_learn.models.simple_mnl import SimpleMNL
from choice_learn.data import ChoiceDataset
from choice_learn.datasets.base import load_heating

Let's recreate this [tutorial](https://cran.r-project.org/web/packages/mlogit/vignettes/e1mlogit.html) by Yves Croissant for the mlogit R package.

It uses the Heating dataset, where we try to predict which heating harware a houseold will chose. The dataset is integrated in the package, you can find information [here].

In [ ]:
heating_df = load_heating(as_frame=True)

contexts_features = ["income", "agehed", "rooms"]
choice = ["depvar"]
contexts_items_features = ["ic.", "oc."]
items = ["hp", "gc", "gr", "ec", "er"]

choices = np.array([items.index(val) for val in heating_df[choice].to_numpy().ravel()])
contexts = heating_df[contexts_features].to_numpy()
contexts_items = np.stack([heating_df[[feat + item for feat in contexts_items_features]].to_numpy() for item in items], axis=1)

First part estimates a simple MNL without intercept from the 'ic' and 'oc' features. By default, SimpleMNL does not integrate any intercept, but you can precise 'None'.

In [ ]:
dataset = ChoiceDataset(contexts_items_features=contexts_items, choices=choices)
model = SimpleMNL(optimizer="lbfgs", intercept=None)
history = model.fit(dataset, epochs=100, get_report=True)

In [ ]:
print("Estimation Negative LogLikelihood:",
      model.evaluate(dataset) * len(dataset))

In [ ]:
model.report

We reach very similar results. The second part is about modelling useing the ic + oc/0.12 ratio. Here is how it can be done:

In [ ]:
ratio_contexts_items = []
for case in range(contexts_items.shape[0]):
    feat = []
    for item in range(contexts_items.shape[1]):
        feat.append([contexts_items[case, item, 0] + contexts_items[case, item, 1] / 0.12])
    ratio_contexts_items.append(feat)
ratio_contexts_items = np.array(ratio_contexts_items)
ratio_contexts_items.shape

In [ ]:
ratio_dataset = ChoiceDataset(contexts_items_features=ratio_contexts_items, choices=choices)
model = SimpleMNL(optimizer="lbfgs")
history = model.fit(ratio_dataset, epochs=100, get_report=False)

In [ ]:
print("Weights:", model.weights)
print("Estimation Negative LogLikelihood:", model.evaluate(ratio_dataset) * len(ratio_dataset))

Finally, to add itemwise intercept for the last part, here is how it can be done:

In [ ]:
model = SimpleMNL(optimizer="lbfgs", intercept="item")
history = model.fit(dataset, epochs=100, get_report=True)

In [ ]:
model.report